# WGAN-gp demo

This notebook contains methods from paper: Improved Training of Wasserstein GANs (https://arxiv.org/abs/1704.00028), which introduces gradient penalty loss and layer normalization in the discriminator network, and improves the training stability and performance of GAN.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [ ]:
(images_train, label), _ = mnist.load_data()
images_train = images_train / 255.0
images_train = images_train[:,:,:,np.newaxis]
images_train.shape

In [ ]:
# ref: https://www.tensorflow.org/tutorials/generative/dcgan
def generator(latent_dim):
    return tf.keras.Sequential([
        tf.keras.layers.Input([latent_dim]),
        tf.keras.layers.Dense(units=(7 * 7 * 256), use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Reshape((7, 7, 256)),
        tf.keras.layers.Conv2DTranspose(
            filters=128,
            kernel_size=5,
            strides=1,
            padding='same',
            use_bias=False,
        ),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Conv2DTranspose(
            filters=64,
            kernel_size=5,
            strides=2,
            padding='same',
            use_bias=False,
        ),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Conv2DTranspose(
            filters=1,
            kernel_size=5,
            strides=2,
            padding='same',
            use_bias=False,
            activation='sigmoid',
        ),
    ])


def discriminator():
    return tf.keras.Sequential([
        tf.keras.layers.Input([28, 28, 1]),
        tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=2,
                               padding='same'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.LeakyReLU(),
        
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=5, strides=2,
                               padding='same'),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.LeakyReLU(),
        
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1),
    ])

In [ ]:
class GAN(tf.keras.Model):

    def __init__(self, generator, discriminator, latent, gp_weight=10, disc_steps=3):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.gp_weight = gp_weight
        self.disc_steps = disc_steps
        self.latent = latent
        
    def gradient_penalty(self, real, fake):
        batch_size = tf.shape(real)[0]
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake - real
        interpolated = real + alpha * diff
        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = self.discriminator(interpolated, training=True)
        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    @staticmethod
    def discriminator_loss(real_logits, fake_logits):
        real_loss = tf.reduce_mean(real_logits)
        fake_loss = tf.reduce_mean(fake_logits)
        return fake_loss - real_loss

    @staticmethod
    def generator_loss(fake_logits):
        return -tf.reduce_mean(fake_logits)
    
    def train_step(self, inputs):
        real = inputs
        
        disc_vars = self.discriminator.trainable_variables
        gen_vars = self.generator.trainable_variables

        for i in range(self.disc_steps): 
            noise = tf.random.normal(shape=[tf.shape(real)[0], self.latent])
            with tf.GradientTape() as disc_tape:
                fake = self.generator(noise, training=True)
                real_logits = self.discriminator(real, training=True)
                fake_logits = self.discriminator(fake, training=True)
                disc_loss = self.discriminator_loss(real_logits, fake_logits)
                disc_loss += self.gp_weight * self.gradient_penalty(real, fake)

            disc_grads = disc_tape.gradient(disc_loss, disc_vars)
            self.optimizer.apply_gradients(zip(disc_grads, disc_vars))

        with tf.GradientTape() as gen_tape:
            noise = tf.random.normal(shape=[tf.shape(real)[0], self.latent])
            fake = self.generator(noise, training=True)
            real_logits = self.discriminator(real, training=True)
            fake_logits = self.discriminator(fake, training=True)
            gen_loss = self.generator_loss(fake_logits)
            
        gen_grads = gen_tape.gradient(gen_loss, gen_vars)
        self.optimizer.apply_gradients(zip(gen_grads, gen_vars))

        return {'gen_loss': gen_loss, 'disc_loss': disc_loss}

In [ ]:
latent_dim = 128
epochs = 20
lr = 2e-4
batch_size = 256
model = GAN(generator=generator(latent_dim), 
            discriminator=discriminator(),
            latent = latent_dim)
model.compile(optimizer=tf.keras.optimizers.Adam(beta_1=0.5, beta_2=0.9, learning_rate=lr))

In [ ]:
model.fit(images_train, epochs=epochs, batch_size=batch_size)

In [ ]:
#display results
def img_grid(imgs, rows, cols, title):
    fig = plt.figure()
    for i, j in np.ndindex(rows, cols):
        plt.subplot(rows, cols, i*cols+j+1)
        plt.imshow(np.repeat(imgs[i*cols+j], 3, -1))
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

cols = 8
rows = 8
inp = images_train[:cols*rows]
gen = model.generator.predict(np.random.normal(size=(cols*rows, latent_dim)))
img_grid(inp, rows, cols, 'Original')
img_grid(gen, rows, cols, 'Generated')